In [48]:
import pandas  as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras

In [49]:
df = pd.read_csv('../Data/LAB 5 data.csv')
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X192,X193,X194,X195,X196,X197,X198,X199,X200,y
0,-0.105716,-1.056372,0.472175,-0.624175,-1.337268,0.109321,0.293656,0.635470,-0.871020,0.194277,...,0.101041,0.708435,1.343249,-1.202234,-1.333481,-0.888842,0.134859,0.355681,-1.193989,180.621931
1,1.255322,0.125270,0.450023,2.003561,0.277094,0.450579,0.462558,-0.316563,0.052639,0.188200,...,-0.638715,0.407518,1.452270,-1.047522,0.697560,0.335719,-2.483643,0.396654,-1.337587,8.336719
2,0.655216,0.825484,-0.890328,0.105923,-0.627933,1.788487,-0.338370,-0.601686,-0.837622,-0.085283,...,0.108930,1.120339,-0.557336,0.542774,-2.099880,0.766930,-0.640871,0.607291,0.593155,178.514922
3,-0.862742,-0.297754,0.316790,-2.747953,-0.468321,1.168713,-0.639897,0.602207,-0.390683,-0.279624,...,1.174023,-0.597059,-0.768229,-1.508851,-1.563351,0.824011,1.152560,0.288155,0.428682,233.401328
4,-0.492418,0.832162,-0.704394,-0.502445,0.734592,-0.435019,-0.017912,-1.992973,-0.129440,1.429077,...,1.123942,2.143922,2.052964,0.224794,1.871673,-0.728549,-1.876174,-1.169126,1.209643,-98.768226


In [50]:
df = df.dropna()
K = 25 # number of top selected features
X = df.drop(columns=['y'])
y = df['y']

In [51]:
# Perform feature selection
selector = SelectKBest(score_func=f_regression, k=K)
X_selected = selector.fit_transform(X, y)
X_selected_df = pd.DataFrame(X_selected, columns=X.columns[selector.get_support(indices=True)])


In [52]:
# Display the selected features
print("Selected Features:")
print(X_selected_df.head())

Selected Features:
         X5        X6       X17       X26       X49       X52       X56  \
0 -1.337268  0.109321  1.685851  0.261508 -0.595003  1.693254 -0.537250   
1  0.277094  0.450579  0.496710 -2.562039  0.904076 -0.914503 -0.070094   
2 -0.627933  1.788487 -1.361414  0.746342 -1.090604 -0.549102  1.308424   
3 -0.468321  1.168713 -0.588841 -0.955397 -0.845406  1.518231  0.296104   
4  0.734592 -0.435019  0.792072  0.283879 -0.418987  1.285036  0.544084   

        X83       X89       X91  ...      X144      X145      X153      X157  \
0 -1.030831  1.639892 -0.235762  ...  0.186388 -0.061207 -0.494624  0.935732   
1 -0.331297  0.121529 -0.326941  ... -0.680156  1.316802  0.648372  0.735678   
2  0.631977 -1.156814  0.579440  ... -1.794573  0.296453 -0.885534  1.029486   
3  0.700889 -1.095730  0.144831  ... -0.275254 -0.115787 -0.365944  1.784879   
4  1.303945  0.620423  0.742086  ... -2.493134 -0.046187 -0.600990  0.415368   

       X161      X166      X167      X181      X1

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_selected_df, y, test_size=0.2, random_state=42)


In [54]:
def create_model(input_dim):
  model = keras.Sequential([
      keras.layers.Dense(32, activation='relu', input_dim=input_dim),
      keras.layers.Dense(16, activation='relu'),
      keras.layers.Dense(8, activation='relu'),
      keras.layers.Dense(1) # no activation --> regression
  ])
  return model

In [55]:
input_dim = X_selected_df.shape[1]
mlp_model = create_model(input_dim)

mlp_model.compile(loss='mean_squared_error', optimizer='adam')
epochs = 100
batch_size = 32

history = mlp_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/100
28/28 [==============================] - 1s 8ms/step - loss: 35485.3555 - val_loss: 32421.1094
Epoch 2/100
28/28 [==============================] - 0s 3ms/step - loss: 35408.1758 - val_loss: 32339.5488
Epoch 3/100
28/28 [==============================] - 0s 3ms/step - loss: 35267.8203 - val_loss: 32167.5488
Epoch 4/100
28/28 [==============================] - 0s 3ms/step - loss: 34966.3086 - val_loss: 31802.4297
Epoch 5/100
28/28 [==============================] - 0s 4ms/step - loss: 34321.6797 - val_loss: 31048.2461
Epoch 6/100
28/28 [==============================] - 0s 7ms/step - loss: 33088.4102 - val_loss: 29727.0312
Epoch 7/100
28/28 [==============================] - 0s 5ms/step - loss: 30977.5820 - val_loss: 27515.5898
Epoch 8/100
28/28 [==============================] - 0s 5ms/step - loss: 27702.1934 - val_loss: 24159.3828
Epoch 9/100
28/28 [==============================] - 0s 5ms/step - loss: 22832.8340 - val_loss: 19152.7578
Epoch 10/100
28/28 [=================

In [56]:
# Evaluate the Model
y_pred = mlp_model.predict(X_test)

# Evaluate the model using appropriate metrics for regression
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

9/9 [==============================] - 0s 2ms/step
Mean Absolute Error (MAE): 7.43346286184345
Mean Squared Error (MSE): 83.93669849055104
R-squared (R2): 0.9976724333482608


# **Justification**


Model Architecture:

The model architecture consists of three hidden layers with 64, 32, and 16 neurons, respectively. This choice of architecture with multiple hidden layers allows the model to learn complex patterns and relationships in the data. Deeper architectures can capture higher-level abstractions, potentially improving performance on complex datasets.

The choice of the ReLU (Rectified Linear Unit) activation function for the hidden layers is justified by its effectiveness in overcoming the vanishing gradient problem, which can occur when using sigmoid or tanh activations. ReLU is computationally efficient and allows the model to learn non-linear mappings effectively.

The output layer has a single neuron without any activation function, which is appropriate for regression tasks. The output of the model is directly used as the predicted value, and there is no need for any activation function to constrain the output range.

Feature Selection Technique:

The feature selection technique used is SelectKBest with f_regression as the score function. SelectKBest selects the top K features based on their correlation with the target variable (y).